In [ ]:
import cv2
import numpy as np
import os

# Load YOLOv4 model
net = cv2.dnn.readNet("yolov4.weights", "yolov4.cfg")
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

# Load the class labels (from coco.names)
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

# Define paths
images_path = "C:/Users/arpit/Downloads/CODdataset/testing_images"  # Input images folder
output_path = "C:/Users/arpit/Downloads/CODdataset/bbox_visualized_test"  # Folder to save visualized images

# Ensure output directory exists
os.makedirs(output_path, exist_ok=True)

# Iterate through all images
for img_file in os.listdir(images_path):
    if img_file.endswith(('.jpg', '.jpeg', '.png')):  # Process valid image files
        img_path = os.path.join(images_path, img_file)

        # Load the image
        img = cv2.imread(img_path)
        height, width, channels = img.shape

        # Prepare the image for YOLO detection
        blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
        net.setInput(blob)
        outs = net.forward(output_layers)

        # Filter for cars (class 2 in COCO dataset corresponds to 'car')
        for out in outs:
            for detection in out:
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]
                if confidence > 0.5 and class_id == 2:  # Class ID 2 corresponds to "car"
                    # Get bounding box coordinates
                    center_x = int(detection[0] * width)
                    center_y = int(detection[1] * height)
                    w = int(detection[2] * width)
                    h = int(detection[3] * height)

                    # Get top-left corner coordinates
                    x = int(center_x - w / 2)
                    y = int(center_y - h / 2)

                    # Draw the bounding box around the detected car
                    cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
                    cv2.putText(img, "Car", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

        # Save or display the image with bounding boxes
        output_img_path = os.path.join(output_path, img_file)
        cv2.imwrite(output_img_path, img)
        print(f"Processed: {img_file}")